In [9]:
import sqlite3
from selenium import webdriver
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from selenium.webdriver.common.by import By
import re, sys
from selenium.common.exceptions import NoSuchElementException
from sqlite3 import IntegrityError, InterfaceError

In [3]:
# 1. 테이블 만들기
conn = sqlite3.connect('HKreports.db')

cur = conn.cursor()

reports_q = "CREATE TABLE reports(reports_idx TEXT, company TEXT NOT NULL, date DATE NOT NULL, written_by TEXT NOT NULL,PRIMARY KEY (reports_idx));"
cur.execute(reports_q)
companies_q = "CREATE TABLE companies(company TEXT NOT NULL, code TEXT UNIQUE NOT NULL, category TEXT, PRIMARY KEY (company));"
cur.execute(companies_q)

[]

In [16]:
# 2. 테이블에 집어넣기
def GetCategory(code):
    try:
        url = f"https://finance.naver.com/item/main.naver?code={code}"
        res = requests.get(url)
        soup = BeautifulSoup(res.text, "html.parser")
        category = soup.select_one('div.trade_compare>h4>em>a').text
    
    except:
        category = "몰?루"
    
    finally:
        return category 



def CollectDatas():
    
    # 한경 컨센서스의 보고서에 올라온 기업과 관련된 정보를 탐색 및 SQL에 데이터 저장.
    
    today = datetime.today().date()
    
    # if db == mysql:
        
    # conn = Initialize() 
        
    # start_date = FindStartDate(conn)
    start_date = "2022-08-01" # 테스트용 
    
    get_name = re.compile(".+?(?=\()") # .+만 쓰면 가장 큰 범위부터 탐색해서 매칭하면 끝남. ?을 붙여 reluctant를 이용한다.
    get_code = re.compile("([0-9]{6})")
    find_idx_code = re.compile('[0-9]+')
    
    # 220817 : Chrome Headless
    opt = webdriver.ChromeOptions()
    opt.add_argument('headless')
    
    conn = sqlite3.connect('HKreports.db')
    
    # with conn.cursor() as curs: # with문 쓰면 sqlite3에서는 에러가 발생함
    curs = conn.cursor()
    with webdriver.Chrome('../chromedriver.exe', chrome_options = opt) as driver:
        driver.implicitly_wait(2)
        
        # print("시작 날짜 : ", start_date, "오늘 날짜 : ", today)
        
        # 페이지 갯수 얻고 시작
        exit_switch = False
        try:
            driver.get(f'http://hkconsensus.hankyung.com/apps.analysis/analysis.list?sdate={start_date}&edate={today}&now_page=1&search_value=&report_type=UP&pagenum=80&search_text=&business_code=')
            pages = driver.find_element(By.CSS_SELECTOR, 'div.paging>a.last').get_attribute('href')
            last_page = re.compile('(?<=now_page\=)[0-9]+').findall(pages)[0] # 후방탐색
        
        except NoSuchElementException: # 아무 데이터가 없다면 이 에러가 뜸 & 강제종료
            print("아무 데이터도 없어 프로그램을 종료함")
            exit_switch = True
            
        if exit_switch:
            sys.exit(0)
        
        for i in range(1, int(last_page) + 1):
            
            if i != 1:
                driver.get(f'http://hkconsensus.hankyung.com/apps.analysis/analysis.list?sdate={start_date}&edate={today}&now_page={i}&search_value=&report_type=UP&pagenum=80&search_text=&business_code=')

            contents = driver.find_elements(By.CSS_SELECTOR, 'table>tbody>tr')
            
            for j in contents:
                # 웹 스크래핑
                try:
                    
                    date = j.find_element(By.CSS_SELECTOR, 'td.txt_number').text
                    report_idx = find_idx_code.findall(j.find_element(By.CSS_SELECTOR, 'td.text_l>a').get_attribute('href'))[0]
                    report = j.find_element(By.CSS_SELECTOR, 'td.text_l').text
                    name = get_name.findall(report)[0]
                    code = get_code.findall(report)[0]
                    
                    category = GetCategory(code) 
                    made_by = j.find_element(By.CSS_SELECTOR, 'td:nth-child(5)').text


                    print(name, code, date, category, made_by, report_idx) 
                    
                    # sqlite3은 %s 대신에 ?를 써야한다
                    query1 = "INSERT INTO reports(company, date, written_by, reports_idx) VALUES(?, ?, ?, ?);"
                    curs.execute(query1, (name, date, made_by, report_idx))

                    query2 = "INSERT INTO companies(company, code, category) VALUES(?, ?, ?);"
                    curs.execute(query2, (name, code, category))
                    
                    
                except InterfaceError:
                    print("인터페이스 에러 발생")

                except IntegrityError:
                    pass
                
                except NoSuchElementException: # 이 예외는 올라온 보고서가 없는 경우에도 작동할 거라고 기대할 수 있겠다
                    break # 이렇게 둬도 반복문 잘 작동함
                
        curs.close()
                    
    conn.commit()
    conn.close()

In [17]:
CollectDatas()

C:\Users\dowra\AppData\Local\Temp\ipykernel_21204\2679052340.py:42: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome('./chromedriver.exe', chrome_options = opt) as driver:
C:\Users\dowra\AppData\Local\Temp\ipykernel_21204\2679052340.py:42: DeprecationWarning: use options instead of chrome_options
  with webdriver.Chrome('./chromedriver.exe', chrome_options = opt) as driver:


JYP Ent. 035900 2022-08-17 방송과엔터테인먼트 SK증권 612887
엘앤에프 066970 2022-08-17 전기제품 대신증권 612883
JYP Ent. 035900 2022-08-17 방송과엔터테인먼트 하이투자증권 612882
엘앤에프 066970 2022-08-17 전기제품 하이투자증권 612869
JYP Ent. 035900 2022-08-17 방송과엔터테인먼트 유진투자증권 612866
하이록코리아 013030 2022-08-17 기계 메리츠증권 612838
태광 023160 2022-08-17 기계 메리츠증권 612836
영원무역 111770 2022-08-17 섬유,의류,신발,호화품 메리츠증권 612821
와이지엔터테인먼트 122870 2022-08-17 방송과엔터테인먼트 이베스트증권 612817
인탑스 049070 2022-08-17 핸드셋 이베스트증권 612811
성광벤드 014620 2022-08-17 기계 메리츠증권 612809
LG 003550 2022-08-16 복합기업 이베스트증권 612750
한국전력 015760 2022-08-16 전기유틸리티 메리츠증권 612715
현대해상 001450 2022-08-15 손해보험 대신증권 612691
네오위즈 095660 2022-08-15 게임엔터테인먼트 대신증권 612683
와이지엔터테인먼트 122870 2022-08-15 방송과엔터테인먼트 SK증권 612678
LG 003550 2022-08-15 복합기업 SK증권 612675
NHN한국사이버결제 060250 2022-08-15 IT서비스 이베스트증권 612661
크래프톤 259960 2022-08-12 게임엔터테인먼트 한화투자증권 612626
CJ프레시웨이 051500 2022-08-11 식품과기본식료품소매 하이투자증권 612605
신세계 004170 2022-08-11 백화점과일반상점 유안타증권 612580
KT 030200 2022-08-11 다각화된통신서비스 메리츠증권 612562
콘텐트리중앙 036420 2022-0

In [26]:
# 3. 테이블 조회하기 

def ReadPreviousWeeks(weeks = 2, read_all = False):

    db = sqlite3.connect('HKreports.db')
    # 일단 메모리 측면에서 생각해보면 당연히 쿼리를 날려서 일부 데이터만 메모리에 올리는 쪽이 훨씬 효율적일 것

    end_date = datetime.today().date()
    
    if read_all:
        start_date = '2022-08-01'
    else:
        start_date = end_date - timedelta(weeks = weeks)
    
    query1 = f"SELECT * FROM reports WHERE date >= '{start_date}' and date <= '{end_date}' ORDER BY date DESC"

#     query1 = (f"SELECT r.company as company, r.date as date, r.written_by as written_by, \
#             c.code as company_code, c.category as category, r.reports_idx as reports_idx\
#             FROM reports r \
#             JOIN companies c \
#             ON r.company = c.company \
#             WHERE date >= '{start_date}' and date <= '{end_date}'")

    query2 = (f"SELECT * \
                FROM companies")

    reports_df = pd.read_sql_query(query1, db)
    reports_df = reports_df.set_index('reports_idx')
    companies_df = pd.read_sql_query(query2, db)
    # reports_df.drop_index()

    db.close()
    
    return reports_df, companies_df
    
df1, df2 = ReadPreviousWeeks()

print(df1)

                company        date written_by
reports_idx                                   
612887         JYP Ent.  2022-08-17       SK증권
612883             엘앤에프  2022-08-17       대신증권
612882         JYP Ent.  2022-08-17     하이투자증권
612869             엘앤에프  2022-08-17     하이투자증권
612866         JYP Ent.  2022-08-17     유진투자증권
612838           하이록코리아  2022-08-17      메리츠증권
612836               태광  2022-08-17      메리츠증권
612821             영원무역  2022-08-17      메리츠증권
612817        와이지엔터테인먼트  2022-08-17     이베스트증권
612811              인탑스  2022-08-17     이베스트증권
612809             성광벤드  2022-08-17      메리츠증권
612750               LG  2022-08-16     이베스트증권
612715             한국전력  2022-08-16      메리츠증권
612691             현대해상  2022-08-15       대신증권
612683             네오위즈  2022-08-15       대신증권
612678        와이지엔터테인먼트  2022-08-15       SK증권
612675               LG  2022-08-15       SK증권
612661       NHN한국사이버결제  2022-08-15     이베스트증권
612626             크래프톤  2022-08-12     한화투자증권
612605       

In [27]:
print(df2)

        company    code      category
0      JYP Ent.  035900     방송과엔터테인먼트
1          엘앤에프  066970          전기제품
2        하이록코리아  013030            기계
3            태광  023160            기계
4          영원무역  111770  섬유,의류,신발,호화품
5     와이지엔터테인먼트  122870     방송과엔터테인먼트
6           인탑스  049070           핸드셋
7          성광벤드  014620            기계
8            LG  003550          복합기업
9          한국전력  015760        전기유틸리티
10         현대해상  001450          손해보험
11         네오위즈  095660      게임엔터테인먼트
12   NHN한국사이버결제  060250         IT서비스
13         크래프톤  259960      게임엔터테인먼트
14      CJ프레시웨이  051500    식품과기본식료품소매
15          신세계  004170      백화점과일반상점
16           KT  030200     다각화된통신서비스
17       콘텐트리중앙  036420     방송과엔터테인먼트
18           웹젠  069080      게임엔터테인먼트
19          NHN  181710         IT서비스
20        나스미디어  089600            광고
21    한화에어로스페이스  012450       우주항공과국방
22          덴티움  145720     건강관리장비와용품
23          카카오  035720    양방향미디어와서비스
24         셀트리온  068270            제약
25         롯